In [69]:
import pandas as pd
import numpy as np
import string
from collections import Counter
from utils import *
from vehicle import Vehicle

import networkx as nx
from bokeh.io import show, output_file, output_notebook
from bokeh.plotting import figure
from bokeh.models.graphs import from_networkx


## Estrazione dati con un piccolo parser dei dati

In [70]:
path='Instances/H5.txt'

In [71]:
#support
count = 0
#first 3 lines of the txt file
number_of_customers=0
one=0
number_of_vehicles=0
#vettori di informazione
vect_cord=[] #lista/vettore di coppiadi coordinate
vect_domanda=[] #linehaul
vect_carico=[] #backhaul
vec_b=[] #vettore delle istanze backhaul
vec_l=[] #vettore delle istanze linehaul
with open(path) as file:
    for line in file:
        #print(line)
        count+=1
        if count  == 1:
            number_of_customers=int((line.split())[0])
        if count  == 2:
            one=int((line.split())[0])
        if count == 3:
            number_of_vehicles=int((line.split())[0])
        if count > 3:
            vect_cord.append((int((line.split())[0]),int((line.split())[1])))
            vect_domanda.append((int((line.split())[2])))
            vect_carico.append((int((line.split())[3]))) 
            if int((line.split())[3]) == 0:
                vec_l.append(count-4) #il numero delle istanze linehaul
            if int((line.split())[3]) > 0:
                vec_b.append(count-4) # il numero delle istanze backhaul

#capacità massima di ogni mezzo
capacity=vect_carico[0]                     

In [72]:
df_data = pd.DataFrame({'': [number_of_customers, number_of_vehicles, capacity]},
                       index=['Number of customers', 'Number of Vehicles', 'Capacity'])

In [73]:
df_instances = pd.DataFrame({'Cordinate X': [ x[0] for x in vect_cord],
                             'Cordinate Y': [ x[1] for x in vect_cord], 
                             'Linehaul': vect_domanda, 
                             'Backhaul' : vect_carico,
                            })

In [74]:
df_data

,
Number of customers,68
Number of Vehicles,4
Capacity,7100


all'indice 0 è riportato il magazzino. dove il campo Backhaul rappresenta la capacità massima

In [75]:
df_instances

,Cordinate X,Cordinate Y,Linehaul,Backhaul
0,12000,16000,0,7100
1,4405,23028,0,1033
2,20807,3179,0,445
3,1595,26792,0,449
4,23808,13919,0,570
5,4385,8499,0,383
6,22935,9891,0,488
7,23316,19678,0,370
8,16691,26705,0,550
9,21138,20720,0,382


## Calcolo dei Dati raccolti

In [78]:
distance = distance_matrix(vect_cord)

In [79]:
saving=saving_matrix(distance)

In [80]:
dict_Saving_Order=savingOrder(saving)

## Mappa dei saving ordinati

In [82]:
dict_b, dict_l , dict_merge= route_Linehaul_Backhaul(dict_Saving_Order, vec_b, vec_l)

## Inizializzo la lista delle rotte

In [84]:
list_route = []
set_iniziale = {0}
set_first = {0}
set_visited = {0}
lista_keys= list(dict_l.keys())
for k in lista_keys:
    if len(list_route) < number_of_vehicles and k[0] not in set_iniziale and k[1] not in set_iniziale:
        list_route.append(Vehicle([0, k[0], k[1]], vect_domanda[k[0]]+vect_domanda[k[1]],
                         distance[0][k[0]]+distance[k[0]][k[1]]))
        set_iniziale.add(k[0])
        set_iniziale.add(k[1])
        set_first.add(k[0])
        set_visited.add(k[0])
        set_visited.add(k[1])

In [85]:
remove_first_instance(dict_l, set_first)

In [89]:
c = 0
set_first = {0}
array    
while(len(dict_l) >(np.math.factorial(number_of_vehicles)/(np.math.factorial(number_of_vehicles-2)*2))):
    lista_keys = list(dict_l.keys())
    flag = True
    for k in lista_keys:
        if list_route[c].route[-1] in k \
        and different(list_route[c].route[-1], k) not in set_visited \
        and list_route[c].weight + vect_domanda[different(list_route[c].route[-1],k)] <= capacity:
                
                list_route[c].add_weight(vect_domanda[different(list_route[c].route[-1],k)])
                list_route[c].add_route(k)           
                set_visited.add(list_route[c].route[-1])
                list_route[c].add_cost(distance[k[0]][k[1]])
                set_first.add(k[0])
                break

    remove_first_instance(dict_l, set_first)
    c = (c + 1) % 4

KeyboardInterrupt: 

In [86]:
for e in list_route:
    print(e.route)

[0, 32, 41]
[0, 34, 50]
[0, 29, 43]
[0, 30, 45]
